In [1]:
import os
os.environ["VLLM_ALLOW_INSECURE_SERIALIZATION"] = "1"
os.environ["VLLM_USE_CUDA_GRAPH"] = "0"

In [2]:
import torch                                                                                                                                                                                                                          
from pathlib import Path
from chatspace.analysis import (
    load_individual_trait_vectors,
)

                                                                                                                                                                                                                                    
TARGET_LAYER = 31
persona_data_root = Path("/workspace/persona-data/qwen-3-32b")
traits_dict = load_individual_trait_vectors(
    persona_data_root / "traits_240" / "vectors",
    layer_idx=TARGET_LAYER
)
vector = traits_dict['acerbic']
vector = vector / torch.linalg.norm(vector, dim=-1, keepdim=True)
print(vector.shape)
print(torch.linalg.norm(vector, dim=-1))


INFO 10-15 20:22:36 [__init__.py:216] Automatically detected platform cuda.
torch.Size([5120])
tensor(1., dtype=torch.bfloat16)


In [ ]:
from chatspace.generation import VLLMSteerModel, VLLMSteeringConfig                                                                                                                                                                   
# 1. Load the base steering model (match the run’s model/config)                                                                                                                                                                      
cfg = VLLMSteeringConfig(                                                                                                                                                                                                             
    model_name="Qwen/Qwen3-32B",                                                                                                                                                                                                     
    target_layer=TARGET_LAYER,
    init_scale=0.0,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.9,
    max_model_len=256,
)
model = VLLMSteerModel(cfg)


INFO 10-15 20:22:38 [utils.py:233] non-default args: {'max_model_len': 256, 'disable_log_stats': True, 'model': 'Qwen/Qwen3-32B'}
INFO 10-15 20:22:38 [model.py:547] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-15 20:22:38 [model.py:1510] Using max model len 256
INFO 10-15 20:22:38 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=16384.
WARNING 10-15 20:22:38 [serial_utils.py:51] Allowing insecure serialization using pickle due to VLLM_ALLOW_INSECURE_SERIALIZATION=1
(EngineCore_DP0 pid=2895489) INFO 10-15 20:22:38 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=2895489) INFO 10-15 20:22:38 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='Qwen/Qwen3-32B', speculative_config=None, tokenizer='Qwen/Qwen3-32B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=256, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredO

Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]


(EngineCore_DP0 pid=2895489) INFO 10-15 20:22:53 [default_loader.py:267] Loading weights took 12.16 seconds
(EngineCore_DP0 pid=2895489) INFO 10-15 20:22:54 [gpu_model_runner.py:2653] Model loading took 61.0347 GiB and 12.602225 seconds
(EngineCore_DP0 pid=2895489) INFO 10-15 20:23:02 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/5dd787f23b/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=2895489) INFO 10-15 20:23:02 [backends.py:559] Dynamo bytecode transform time: 8.37 s
(EngineCore_DP0 pid=2895489) INFO 10-15 20:23:06 [backends.py:164] Directly load the compiled graph(s) for dynamic shape from the cache, took 2.918 s
(EngineCore_DP0 pid=2895489) INFO 10-15 20:23:07 [monitor.py:34] torch.compile takes 8.37 s in total
(EngineCore_DP0 pid=2895489) INFO 10-15 20:23:09 [gpu_worker.py:298] Available KV cache memory: 58.86 GiB
(EngineCore_DP0 pid=2895489) INFO 10-15 20:23:10 [kv_cache_utils.py:1087] GPU KV cache size: 241,088 tokens
(EngineCor

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:05<00:00, 11.95it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 67/67 [00:04<00:00, 16.47it/s]


(EngineCore_DP0 pid=2895489) INFO 10-15 20:23:20 [gpu_model_runner.py:3480] Graph capturing finished in 10 secs, took 2.41 GiB
(EngineCore_DP0 pid=2895489) INFO 10-15 20:23:20 [core.py:210] init engine (profile, create kv cache, warmup model) took 26.05 seconds
INFO 10-15 20:23:21 [llm.py:306] Supported_tasks: ('generate',)


In [ ]:

# 3. Apply target strength
strength = 200_000
model.set_vector(vector * strength)

# 4. Run a probe prompt
outputs = model.generate(
    ["Explain why pineapple pizza is controversial."],
    temperature=0.7,
    max_tokens=120,
)
print(outputs[0])

 Pineapple pizza, often referred to as "Hawaiian pizza," is a culinary topic that sparks strong opinions and debates. Here's a structured explanation of why it's controversial:

1. **Cultural and Traditional Preferences**: 
   - **Traditional Italian Pizza**: In Italy, pizza is typically made with simple, high-quality ingredients like tomatoes, mozzarella, and fresh basil. The addition of pineapple (a tropical fruit) and ham (a cooked meat) is seen as a deviation from these traditional norms. For many Italians and purists, this combination is an affront to the integrity of pizza


In [5]:
  # Grab the worker copies
  worker_vecs = model._fetch_worker_vectors()
  print(len(worker_vecs))

  # Inspect stats from rank 0
  w0 = worker_vecs[0].to(torch.float32)
  print("abs max:", w0.abs().max().item())
  print("L2 norm:", torch.linalg.norm(w0).item())
  print("Any NaNs?", torch.isnan(w0).any().item())

1
abs max: 89600.0
L2 norm: 199849.09375
Any NaNs? False
